# Natural Language Processing 
**Project Summary**: This project intended to explore what makes an online post popular. Here are the steps of this project.
1. Web Scraping and Download All the articles.
2. Detect Topics using LDA model
3. Try various Machine Learning Algorithms to predict the view of post 

In [1]:
import pandas as pd


In [3]:
online_news = pd.read_csv("/Users/wenxuanzhang/Local Doc/Documents/Work/UM/OnlineNewsPopularity.csv")


In [30]:

from bs4 import BeautifulSoup
import requests
import urllib2

In [31]:
test = online_news["url"][1]
page = urllib2.urlopen(test)

In [13]:
r = requests.get(test)

In [32]:
soup = BeautifulSoup(r.content,"html.parser")
soup2 = BeautifulSoup(page,"html.parser")

In [25]:
online_news["url"][1]

'http://mashable.com/2013/01/07/ap-samsung-sponsored-tweets/'

In [29]:
type(r)

requests.models.Response

In [36]:
name_box = soup.find(section, attrs={'class': 'article-content'})


NameError: name 'section' is not defined

In [45]:
print(soup.prettify())


<!DOCTYPE html>
<html data-env="production" lang="en" xml:lang="en">
 <head>
  <script type="text/javascript">
   window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"faaaef6476","applicationID":"1135420","transactionName":"cAkNR0ENXQpXF0tJDUASEBxACl4R","queueTime":0,"applicationTime":45,"agent":""}
  </script>
  <script type="text/javascript">
   (window.NREUM||(NREUM={})).loader_config={xpid:"VwYHUF9ACQIDUVVQBw=="};window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var o=e[n]={exports:{}};t[n][0].call(o.exports,function(e){var o=t[n][1][e];return r(o||e)},o,o.exports)}return e[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<n.length;o++)r(n[o]);return r}({1:[function(t,e,n){function r(t){try{c.console&&console.log(t)}catch(e){}}var o,i=t("ee"),a=t(19),c={};try{o=localStorage.getItem("__nr_flags").split(","),console&&"function"==typeof console.log&&(c.console=!0,o.indexO

In [57]:
a =  soup.find_all('p')
  
text = a[1].get_text()
text2 = a[2].get_text()

In [62]:

art = [x.get_text() for x in a]

[u'The Associated Press is the latest news organization to experiment with trying to make money from Twitter by using its feed to advertise for other companies. ',
 u'The AP announced Monday that it will share sponsored tweets from Samsung throughout this week for the International CES taking place in Las Vegas. The news service will let Samsung post two tweets per day to the AP\'s Twitter account, which has more than 1.5 million users, and each of these tweets will be labeled "SPONSORED TWEETS."',
 u'This marks the first time that the AP has sold advertising on its Twitter feed, and the company says it spent months developing guidelines to pave the way for this and other new media business models. ',
 u'For this particular promotion, Samsung will provide the sponsored tweets and non-editorial staff at the AP will handle the publishing side. In this way, the company hopes to maintain a clear dividing line between its editorial and advertising operations on Twitter.',
 u'"We are thrille